In [1]:
# importing required python libraries for data cleaning and EDA
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Loading the specific dataset from the Excel workbook.
data = pd.read_excel("Pharm-Data.xlsx", sheet_name = "Data")

FileNotFoundError: [Errno 2] No such file or directory: 'Pharm-Data.xlsx'

In [ ]:
# dataset into a dataframe.
df = pd.DataFrame(data)
df.head()

In [ ]:
df.shape

#### the dataset has 254,082 rows and 18 columns

In [ ]:
df.info()

In [ ]:
# Change Quantity datatype from float to int.
df["Quantity"] = df["Quantity"].astype(int)

In [ ]:
# Checking for duplicates
df.duplicated().sum()

In [ ]:
# Dropping duplicates
df.drop_duplicates(inplace = True)

In [ ]:
#checking for null values
df.isnull().sum()

#### There are no null values in the dataframe

In [ ]:
df.columns

#### The dataframe contains 18 columns which are 'Distributor', 'Customer Name', 'City', 'Country', 'Latitude', 'Longitude', 'Channel', 'Sub-channel', 'Product Name', 'Product Class', 'Quantity', 'Price', 'Sales', 'Month', 'Year', 'Name of Sales Rep', 'Manager', 'Sales Team'. 

In [ ]:
df.describe()


#### It appears that there are negative values for in the sales and Quantity. This maybe the quantity of products returned and the negative sales the cost implication of the returns.

In [ ]:
# Feature Engineering
negative_sales_and_Quantity = df[["Quantity", "Sales"]] < 0
negative_sales_and_Quantity.head()

In [ ]:
df["Product_status"] = np.where( df["Quantity"] < 0, "Returned", "Accepted")
df.head()

In [ ]:
dir(np)

In [ ]:
(df[["Quantity","Sales"]] < 0).sum()

#### when Quantity field had a negative value, so does Sales field.

In [ ]:
# Checking if the Quantity field had values = 0
(df["Quantity"] == 0).any()

In [ ]:
# Checking if the Sales field had values = 0
(df["Sales"] == 0).any()

In [ ]:
# Subsetting the dataframe for observations without zero values in the Sales and Quantity fields
df_clean = df[((df["Quantity"] != 0) & (df["Sales"] != 0))]
df_clean.head()

## Exploratory Data Analysis

In [ ]:
df_clean.columns

In [ ]:
# Number of sales
df_clean["Sales"].count()

#### There are 254,050 sales recorded.

In [ ]:
# Average sales per customer
avg_sales_per_cust = df_clean["Sales"].sum()/ df_clean["Customer Name"].nunique()
print(f"The Average Sales per Customer is ${round(avg_sales_per_cust, 2)}")

#### The Average Sales per Customer is $15, 711,022.28

In [ ]:
# Average Sales in Period
avg_sales_in_period = df_clean["Sales"].mean()
print(f"The Average Sales in Period is ${round(avg_sales_in_period, 2)}")

#### The Average Sales in Period is $46,443.53

In [ ]:
df_clean['Distributor'].nunique()

#### The Manufacturing company has 29 Distributors partnering with them

In [ ]:
df_clean['Customer Name'].nunique()

#### There are 751 Customers purchasing the Pharmacy Products

In [ ]:
df['City'].nunique()

#### Customers are distributed over 749 Cities

In [ ]:
print(df['Country'].nunique())
print(df['Country'].unique())

#### The products of the Pharmacy company are distributed in 2 countries. These countries are Poland and Germany

In [ ]:
print(df['Channel'].nunique())
print(df['Channel'].unique())

#### Products are distributed via two channels: Hospital and Pharmacy.

In [ ]:
print(df["Sub-channel"].nunique())
print(df["Sub-channel"].unique())

#### Product distribution  Sub-channel are Private, Retail, Institution and Government.

In [ ]:
df['Product Name'].nunique()

#### The Pharmacy company produces 240 products.

In [ ]:
print(df['Product Class'].nunique())
print(df['Product Class'].unique())

#### These Products are under 6 product classes or categories: Mood Stabilizers, Antibiotics, Analgesics, Antiseptics, Antipiretics and Antimalarial

In [ ]:
print(df['Sales Team'].nunique())
print(df['Sales Team'].unique())

#### There are 4 Sales Team: Delta, Bravo, Alfa and Charlie

In [ ]:
# Checking for the proportion of returned products
prop_of_returned_products = df["Product_status"].value_counts(normalize = True)
prop_of_returned_products

In [ ]:
# Bar chart of counts of returned and accepted products
prop_of_returned_products.plot(kind = "bar", title = "Proportion of Returned Products")
plt.xlabel("Product Status")
plt.ylabel("Proportion of Products");

#### Less than 1% of products were returned.

#### Both Quantity and Sales field contain values which are equals to zero.

In [ ]:
df_clean.columns

In [ ]:
df_clean.head()

In [ ]:
# What is total sale
total_sales = round(df_clean["Sales"].sum(), 2)
print(f"Total Sales of the Pharmacy Company: ${total_sales}")

#### Total Sales of the Pharmacy Company: $11,798,977,742.2

In [ ]:
Total_sales_and_quantity_by_year = (
    df_clean.groupby("Year").agg(
        Total_sales = ("Sales", "sum"),
        Total_Quantity = ("Quantity", "sum")
    )
)
print(Total_sales_and_quantity_by_year.sort_values(by="Total_sales", ascending = False))

#### Most quantity were sold and Most sales was made in the year 2018.

In [ ]:
Total_sales_and_quantity_by_country = (
    df_clean.groupby("Country").agg(
        Total_sales = ("Sales", "sum"),
        Total_quantity = ("Quantity", "sum")
    )
)
print(Total_sales_and_quantity_by_country.sort_values(by="Total_sales", ascending = False))

#### Most sales were made in Germany and likewise most quantity was sold there

In [ ]:
Total_sales_and_quantity_by_channel = (
    df_clean.groupby("Channel").agg(
        Total_sales = ("Sales", "sum"),
        Total_quantity = ("Quantity", "sum")
    )
)
print(Total_sales_and_quantity_by_channel.sort_values(by="Total_sales", ascending = False))


In [ ]:
Country_Channel_Sales_pivot_table = pd.pivot_table(
    data=df_clean, 
    columns= "Channel", 
    index= "Country", 
    values= ["Sales"],
    aggfunc = "sum")
Country_Channel_pivot_table

In [ ]:
sns.heatmap(Country_Channel_Sales_pivot_table)
plt.title("Heatmap of Sales by Country and Channel")
plt.xlabel("Channel");

In [ ]:
dir(sns)

In [ ]:
Country_Channel_Quantity_pivot_table = pd.pivot_table(
    data=df_clean, 
    columns= "Channel", 
    index= "Country", 
    values= ["Quantity"],
    aggfunc = "sum")
Country_Channel_Quantity_pivot_table

In [ ]:
sns.heatmap(Country_Channel_Quantity_pivot_table)
plt.title("Heatmap of Sales by Country and Channel")
plt.xlabel("Channel");

In [ ]:
Total_sales_and_quantity_by_distributor = (
    df_clean.groupby("Distributor").agg(
        Total_sales = ("Sales", "sum"),
        Total_Quantity = ("Quantity", "sum")
    )
)
top_ten_distributors = (Total_sales_and_quantity_by_distributor.sort_values(by="Total_sales", ascending = False)).head(10)
top_ten_distributors

#### Top ten Distributors

In [ ]:
bottom_ten_distributors = (Total_sales_and_quantity_by_distributor.sort_values(by="Total_sales", ascending = False)).tail(10)
bottom_ten_distributors

#### Bottom ten Distributors

In [ ]:
df_clean.columns

In [ ]:
Total_sales_and_quantity_by_customer = (
    df_clean.groupby("Customer Name").agg(
        Total_sales = ("Sales", "sum"),
        Total_Quantity = ("Quantity", "sum")
    )
)
top_ten_customers = (Total_sales_and_quantity_by_customer.sort_values(by="Total_sales", ascending = False)).head(10)
top_ten_customers

#### Top ten customers 

In [ ]:
Total_sales_and_quantity_by_city = (
    df_clean.groupby(["Country", "City"]).agg(
        Total_sales = ("Sales", "sum"),
        Total_Quantity = ("Quantity", "sum")
    )
)
top_ten_cities = (Total_sales_and_quantity_by_city.sort_values(by="Total_sales", ascending = False)).head(10)
top_ten_cities

#### All the top ten cities are in Germany

In [ ]:
Total_sales_and_quantity_by_product_class = (
    df_clean.groupby("Product Class").agg(
        Total_sales = ("Sales", "sum"),
        Total_Quantity = ("Quantity", "sum")
    )
)
sales_quantity_by_product_class = (Total_sales_and_quantity_by_product_class.sort_values(by="Total_sales", ascending = False))
sales_quantity_by_product_class

In [ ]:
sales_quantity_by_product_class["Total_sales"].plot(kind="bar", title= "Total Sales by Product Class");

In [ ]:
sales_quantity_by_product_class["Total_Quantity"].plot(kind="bar", title= "Total Quantity by Product Class");

#### Analgesic and Antiseptics were the top performing product class

In [ ]:
Total_sales_and_quantity_by_product_name = (
    df_clean.groupby("Product Name").agg(
        Total_sales = ("Sales", "sum"),
        Total_Quantity = ("Quantity", "sum")
    )
)
top_ten_products = (Total_sales_and_quantity_by_product_name.sort_values(by="Total_sales", ascending = False)).head(10)
top_ten_products

In [ ]:
bottom_ten_products = (Total_sales_and_quantity_by_product_name.sort_values(by="Total_sales", ascending = False)).tail(10)
bottom_ten_products

In [ ]:
Total_sales_quantity_by_status = df.groupby("Product_status").agg(
    total_sales = ("Sales", "sum"),
    total_quantity = ("Quantity", "sum")
)
Total_sales_quantity_by_status

In [ ]:
total_returned_product = df[df["Product_status"] == "Returned"]["Quantity"].sum()

In [ ]:
total_returned_product = df[df["Product_status"] == "Returned"]["Quantity"].sum()

total_accepted_product = df[df["Product_status"] == "Accepted"]["Quantity"].sum()

print(f"Total Quantity of Product Returned: {total_returned_product}")
print(f"Total Quantity of Product Accepted: {total_accepted_product}")

In [ ]:
real_quantity_sold = total_accepted_product - (-total_returned_product)
print(f"The True Quantity of Products Sold is {real_quantity_sold}")

#### The True Quantity of Products Sold is 28,678,699

In [ ]:
total_returned_product_cost = df[df["Product_status"] == "Returned"]["Sales"].sum()

total_accepted_product_sales = df[df["Product_status"] == "Accepted"]["Sales"].sum()
print(f"Total Cost of Product Returned: ${total_returned_product_cost}")
print(f"Total Sales of Product Accepted: ${round(total_accepted_product_sales, 2)})")

In [ ]:
real_sales_amount = total_accepted_product_sales - (-total_returned_product_cost)
print(f"The True Sales of Products Amount is ${round(real_sales_amount, 2)}")

#### The True Sales of Products Amount is $11,798,977,742.2
​

In [ ]:
# Saving clean Dataset
df_clean.to_csv("py-cleaned-pharmacy-data.csv", index= False)